In [11]:
!pip install tqdm
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [12]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision
import os
import torch
from PIL import Image
import tqdm

In [4]:
class ImageFolderWithLabels(Dataset):
    def __init__(self, root_dir, transform=None, half_size=False):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self._get_image_paths_and_labels(half_size)

    def _get_image_paths_and_labels(self, half_size=False):
      for subfolder in ["real", "fake"]:
            subfolder_path = os.path.join(self.root_dir, subfolder)
            image_list = os.listdir(subfolder_path)  # Get all filenames

            if half_size:
                image_list = image_list[::2]  # Select every alternate image

            for filename in image_list:
                if filename.endswith(".jpg"):
                    img_path = os.path.join(subfolder_path, filename)
                    self.image_paths.append(img_path)
                    self.labels.append(0 if subfolder == "real" else 1)  # Assign label

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]

        img = Image.open(img_path).convert('RGB')
        if self.transform:
            img = self.transform(img)

        return img, label

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224
    transforms.ToTensor(),  # Convert to PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize using ImageNet statistics
])

In [6]:
# Load data from each directory
train_data = ImageFolderWithLabels(os.path.join('/content/drive/MyDrive/faceforensics_processed_split', 'Train'), transform=transform)#, half_size=True)
test_data = ImageFolderWithLabels(os.path.join('/content/drive/MyDrive/faceforensics_processed_split', 'Test'), transform=transform)#, half_size=True)
val_data = ImageFolderWithLabels(os.path.join('/content/drive/MyDrive/faceforensics_processed_split', 'val'), transform=transform)#, half_size=True)

In [7]:
print(len(train_data))
print(len(test_data))
print(len(val_data))

2048
255
257


In [8]:
BATCH_SIZE=32

In [9]:
# Create dataloaders for training, testing, and validation
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)  # Adjust batch size as needed
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=False)

In [15]:
deit = torch.hub.load('facebookresearch/deit:main', 'deit_tiny_patch16_224', pretrained=True)

Using cache found in /root/.cache/torch/hub/facebookresearch_deit_main


In [17]:
import torch.nn as nn

In [18]:
deit.head = nn.Linear(deit.head.in_features, 2)
for param in deit.head.parameters():
    param.requires_grad = True

In [ ]:
#convnext = torchvision.models.convnext_tiny(weights='DEFAULT')

Downloading: "https://download.pytorch.org/models/convnext_tiny-983f1562.pth" to /root/.cache/torch/hub/checkpoints/convnext_tiny-983f1562.pth
100%|██████████| 109M/109M [00:02<00:00, 44.9MB/s]


In [19]:
total_params = sum(p.numel() for p in deit.parameters())
total_params

5524802

In [20]:
import torch.optim as optim
optimizer = optim.Adam(deit.parameters(), lr=0.001)

In [21]:
# Define your loss function
criterion = nn.CrossEntropyLoss()

In [22]:
# Define the device to run the model on (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [23]:
deit.to(device)

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 192, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=192, out_features=576, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=192, out_features=192, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=192, out_features=768, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity()


In [24]:
def train_model(model, train_loader, val_loader, criterion, optimizer, device, num_epochs=8):
    model.train()  # Set the model to training mode
    for epoch in range(num_epochs):
        # Training phase
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()  # Zero the parameter gradients
            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, labels)  # Calculate the loss
            loss.backward()  # Backward pass
            optimizer.step()  # Optimize
            running_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        # Calculate training loss and accuracy
        epoch_train_loss = running_loss / len(train_loader.dataset)
        epoch_train_acc = correct / total

        # Validation phase
        model.eval()  # Set the model to evaluation mode
        val_running_loss = 0.0
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for val_inputs, val_labels in val_loader:
                val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)
                val_outputs = model(val_inputs)
                val_loss = criterion(val_outputs, val_labels)
                val_running_loss += val_loss.item() * val_inputs.size(0)
                _, val_predicted = val_outputs.max(1)
                val_total += val_labels.size(0)
                val_correct += val_predicted.eq(val_labels).sum().item()

        # Calculate validation loss and accuracy
        epoch_val_loss = val_running_loss / len(val_loader.dataset)
        epoch_val_acc = val_correct / val_total

        # Print epoch statistics
        print(f"Epoch {epoch + 1}/{num_epochs}",
              f"Train Loss: {epoch_train_loss:.4f}, Train Acc: {epoch_train_acc:.4f}",
              f"Val Loss: {epoch_val_loss:.4f}, Val Acc: {epoch_val_acc:.4f}")

    print('Training complete')


num_epochs = 10


train_model(deit, train_loader, val_loader, criterion, optimizer, device, num_epochs)


Epoch 1/10 Train Loss: 0.7143, Train Acc: 0.6494 Val Loss: 0.4592, Val Acc: 0.8016
Epoch 2/10 Train Loss: 0.4866, Train Acc: 0.7622 Val Loss: 0.3415, Val Acc: 0.8794
Epoch 3/10 Train Loss: 0.3528, Train Acc: 0.8540 Val Loss: 0.2647, Val Acc: 0.9027
Epoch 4/10 Train Loss: 0.2294, Train Acc: 0.9102 Val Loss: 0.1604, Val Acc: 0.9377
Epoch 5/10 Train Loss: 0.2169, Train Acc: 0.9150 Val Loss: 0.1968, Val Acc: 0.9027
Epoch 6/10 Train Loss: 0.1803, Train Acc: 0.9287 Val Loss: 0.1160, Val Acc: 0.9533
Epoch 7/10 Train Loss: 0.1610, Train Acc: 0.9424 Val Loss: 0.2135, Val Acc: 0.9222
Epoch 8/10 Train Loss: 0.1400, Train Acc: 0.9492 Val Loss: 0.3501, Val Acc: 0.8482
Epoch 9/10 Train Loss: 0.1151, Train Acc: 0.9556 Val Loss: 0.0568, Val Acc: 0.9767
Epoch 10/10 Train Loss: 0.1053, Train Acc: 0.9600 Val Loss: 0.0712, Val Acc: 0.9767
Training complete


In [25]:
def test_model(model, test_loader, criterion, device):
    model.eval()  # Set the model to evaluation mode
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():  # No need to calculate gradients during testing
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, labels)  # Calculate the loss
            running_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    test_loss = running_loss / len(test_loader.dataset)
    test_acc = correct / total
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")


In [26]:
test_model(deit, test_loader, criterion, device)

Test Loss: 0.1506, Test Accuracy: 0.9490


In [27]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def evaluate_model(model, test_loader, device):
    model.eval()  # Set the model to evaluation mode
    predictions = []
    true_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    # Compute metrics
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions)
    recall = recall_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions)
    confusion_mat = confusion_matrix(true_labels, predictions)

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print("Confusion Matrix:")
    print(confusion_mat)

#Assuming you have already defined your test_loader and model
evaluate_model(deit, test_loader, device)

Accuracy: 0.9490
Precision: 0.9516
Recall: 0.9440
F1 Score: 0.9478
Confusion Matrix:
[[124   6]
 [  7 118]]
